In [ ]:
import sys  
sys.path.insert(0, '/Users/johanneswidera/Uni/bachelorarbeit/Code/models/')

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import pandas as pd
import numpy as np
from sklearn import linear_model
from bs4 import BeautifulSoup
import contractions
import os
import re
import string
from helper import load_csv
from model_helper.tf_idf import build_tf_idf
from model_helper.PipelineWrapper import PipelineWrapper
from custom_shap_explainer.custom_global import custom_global_explanation, custom_global_shap_distribution
from custom_shap_explainer.signal_words import highlight_signal_words
np.random.seed(1337)
shap.initjs()

# 1. Get Training Data 

In [ ]:
def preprocessor(text):
  text = text.lower()
  text = contractions.fix(text)
  text = text.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
  text = re.sub(' +', ' ', text)
  return text 

def load_data(source):
  data = pd.read_csv(source)
  # select only 2 columns
  data = data[['candidate', 'label']]
  # rename columns
  data.columns = ['text', 'label']
  data['text'] = data['text'].apply(preprocessor)
  return data['text'].tolist(), data['label'].tolist()

In [ ]:
corpus_train, y_train = load_data('../data/train.csv')
corpus_test, y_test = load_data('../data/test.csv')

# 2. Build Vectorizer

In [ ]:
vectorizer_tf_idf = build_tf_idf()
vectorizer_tf_idf.fit(corpus_train)

# 3. Build Models

# 3.1 Build Logistic Regression Model

In [ ]:
model = linear_model.LogisticRegression(penalty="l2")
model_logregression_new = PipelineWrapper(model, vectorizer_tf_idf, corpus_test, corpus_train, y_test, y_train)

In [ ]:
model_logregression_new.fit()

In [ ]:
model_logregression_new.report()

In [ ]:
predictions_logreg = model_logregression_new.predict_proba(corpus_test)


LOGREG_PREDICTION_FILE = 'predictions_logreg.csv'

file_exists = os.path.exists(LOGREG_PREDICTION_FILE)

if not file_exists:
    # Calculate BERT predictions
    
    predictions_logreg = model_logregression_new.predict_proba(corpus_test)

# Destructure probabilities for class_1 and class_2
    predictions_class_0 = predictions_logreg[:, 0]
    predictions_class_1 = predictions_logreg[:, 1]
    
    # Create a DataFrame containing the test samples and BERT predictions
    results = pd.DataFrame({
    'logreg_0': predictions_class_0,
    'logreg_1': predictions_class_1
    })
    # Create a DataFrame containing the test samples and BERT predictions
  
    # Save the DataFrame to a CSV file
    results.to_csv(LOGREG_PREDICTION_FILE)

# 3.2 Build Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(min_samples_leaf=20)
model_dtc = PipelineWrapper(model, vectorizer_tf_idf, corpus_test, corpus_train, y_test, y_train)

In [ ]:
model_dtc.fit()

In [ ]:
model_dtc.report()

In [ ]:
print(model_dtc.predict_proba([corpus_test[0]]))

In [ ]:
DTC_PREDICTION_FILE = 'predictions_dtc.csv'

file_exists = os.path.exists(DTC_PREDICTION_FILE)

if not file_exists:
    # Calculate DTC predictions
    predictions_DTC = model_dtc.predict_proba(corpus_test)

# Destructure probabilities for class_1 and class_2
    predictions_class_0 = predictions_DTC[:, 0]
    predictions_class_1 = predictions_DTC[:, 1]
    
    # Create a DataFrame containing the test samples and BERT predictions
    results = pd.DataFrame({
    'dtc_0': predictions_class_0,
    'dtc_1': predictions_class_1
    })
    results.to_csv(DTC_PREDICTION_FILE)

# 3.3 Build BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("../../FineTunedBERT/Argument/26062217")
# Load model
loaded_model = AutoModelForSequenceClassification.from_pretrained("../../FineTunedBERT/Argument/26062217")
model_bert = pipeline('text-classification', model=loaded_model, tokenizer=tokenizer, max_length=512, truncation=True, top_k=None) 

In [ ]:
BERT_PREDICTION_FILE = 'prediction_BERT.csv'

file_exists = os.path.exists(BERT_PREDICTION_FILE)

if not file_exists:
    # Calculate BERT predictions
    raw_predictions_bert = model_bert.predict(corpus_test)

    # Destructure probabilities for class_1 and class_2
    predictions_class_0 = [pred[0]['score'] if pred[0]['label'] == 'LABEL_0' else pred[1]['score'] for pred in raw_predictions_bert]
    predictions_class_1 = [pred[0]['score'] if pred[0]['label'] == 'LABEL_1' else pred[1]['score'] for pred in raw_predictions_bert]

    # Create a DataFrame containing the test samples and BERT predictions
    results = pd.DataFrame({
        'bert_0': predictions_class_0,
        'bert_1': predictions_class_1
    })

    # Save the DataFrame to a CSV file
    results.to_csv(BERT_PREDICTION_FILE)

# 4 Analyze With SHAP

We know that the Logistic Regression Performs better than the Decision Tree Classifier.
To further investigate why thats the case we need to look into the models.

In the following i will do 3 things.

Per Model:
1. Most Positive/Negative Words
2. Investigate the Most Wrong Positive and Negative prediction

Model Comparision:
1. Investigate biggest Prediction gap accross the models

## what is a claim?
The guidelines provided to the annotators present mainly 3 criteria, which all have to be met for a positive label.

1. The sentence must clearly support or contest the topic, and not simply be neutral.
2. It has to be coherent and stand mostly on its own.
3. It has to be convincing, something you could use to sway someone's stance on the topic (a claim is not enough, it has to be backed up).

In [ ]:
masker = shap.maskers.Text(tokenizer=r"\W+") # this will create a basic whitespace tokenizer
# explainer_logreg = shap.Explainer(model_logregression_new.predict_proba , masker)
explainer_logreg = shap.Explainer(model_logregression_new.predict_proba, masker)


In [ ]:
# Performance Optimazation

import os.path
import pickle

shap_values_file = "shap_values_logreg.pkl"

# Überprüfen, ob die Datei existiert
if not os.path.exists(shap_values_file):
    # Berechnen Sie die SHAP-Werte, wenn die Datei nicht existiert
    shap_values_logreg = explainer_logreg(corpus_test)

    # Speichern Sie die SHAP-Werte in einer Datei
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values_logreg, f)
else:
    # Laden Sie die SHAP-Werte aus der Datei, wenn sie existiert
    with open(shap_values_file, "rb") as f:
        shap_values_logreg = pickle.load(f)

### 4.1 Investigate the biggest misclassification


In [ ]:
from Sentiment.disagreements import get_misclassifications
logreg_misclassifications = get_misclassifications(y_test,LOGREG_PREDICTION_FILE)

In [ ]:

print("total samples", len(y_test))
print("total misclassifications", len(logreg_misclassifications))

# count number of samples for label 1 and label 0
label_0 = y_test.count(0)
label_1 = y_test.count(1)



# print number of samples for label 1 and label 0

print("Number of samples for label 0: ", label_0)
print("Number of samples for label 1: ", label_1)

# count missclassifications for class 0 and for class 1

logreg_misclassifications_0 = logreg_misclassifications[(logreg_misclassifications['label'] == 0)]
logreg_misclassifications_1 = logreg_misclassifications[(logreg_misclassifications['label'] == 1)]

# print the number of missclassifications for class 0 and for class 1
print("Number of missclassifications for class 0: ", len(logreg_misclassifications_0))
print("Number of missclassifications for class 1: ", len(logreg_misclassifications_1))


# calculate the percentage of missclassifications for class 0 and for class 1
percentage_0 = len(logreg_misclassifications_0) / label_0
percentage_1 = len(logreg_misclassifications_1) / label_1

# print the percentage of missclassifications for class 0 and for class 1
print("Percentage of missclassifications for class 0: ", percentage_0)
print("Percentage of missclassifications for class 1: ", percentage_1)

In [ ]:
# find first with label = 0
logreg_misclassifications[:5]

In [ ]:
most_wrong_positive_index = 797

#### 4.1.2 Most Wrong Positive Classification

Most wrong review 

In [ ]:
print(logreg_misclassifications.loc[most_wrong_positive_index])
print(corpus_test[most_wrong_positive_index])

In [ ]:
explanation = shap_values_logreg[most_wrong_positive_index, :, 1]

so its 0.5 behind the real label.
But Why lets investigate

In [ ]:
from custom_shap_explainer.signal_words import highlight_signal_words
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

highlight_signal_words(explanation, round_shap_values=3, top_words=4)

### interpretation:

This is a claim but the model is not able to capture the classification correctly. Because it weights the no-claim words (like: is its is) more than the claim words

#### 4.1.2 Most Wrong Negative Classification

Most wrong review 

In [ ]:
# find first with label = 0
most_wrong_negative_index = logreg_misclassifications[logreg_misclassifications['label'] == 0].index[0]

In [ ]:
print(logreg_misclassifications.loc[most_wrong_negative_index])
print(corpus_test[most_wrong_negative_index])

In [ ]:
explainer = shap_values_logreg[most_wrong_negative_index, : , 1]

In [ ]:
shap.plots.text(explainer)
print(len(explainer))
shap.plots.waterfall(explainer)
highlight_signal_words(explainer,round_shap_values=3,top_words=5)

# Interpretation:

THis is a neutral observation ergo no claim but the words:
- right
- libertarian
- ref
- prostituion
- supports
- bear

have high positive impact on the classification.

Furthermore the model is not really learning why this is not a claim it dont knows that 

# 4.2 Global Interpretation Most Positive and negative Words

To understand how the model works

In [ ]:
custom_global_explanation(shap_values_logreg[:,:,1], num_words=10)

In [ ]:
custom_global_shap_distribution(shap_values_logreg[:,:,1], threshold=0.02)

## Interpretation

The most important Words Contributing to a Claim Classification:

1. opposes
2. ban
3. supports
4. opposed

with the highest shap value of 0.2 of oppeses

The most important Words Contributing to a no-claim classification:

1. Private
2. free 
3. include
4. other

with the highest shap value of -0.1 

you can see that the model pays more attention to the words that are characteristic for a claim classification and less to words that are non-claim charateristic.


but maybe as soon as the main claim classification words are not present it struggles to identify a sample as claim.


This fits to our previous observation were i looked for the misclassified classes and the biggest misclassifications.
Percentage of missclassifications for class 0:  0.030917874396135265
Percentage of missclassifications for class 1:  0.787701317715959

# 5. Analyze with SHAP: DTC

In [ ]:
masker = shap.maskers.Text(tokenizer=r"\W+") # this will create a basic whitespace tokenizer
# explainer_logreg = shap.Explainer(model_logregression_new.predict_proba , masker)
explainer_dtc = shap.Explainer(model_dtc.predict_proba, masker)

In [ ]:
# Performance Optimazation

import os.path
import pickle

shap_values_file = "shap_values_dtc.pkl"

# Überprüfen, ob die Datei existiert
if not os.path.exists(shap_values_file):
    # Berechnen Sie die SHAP-Werte, wenn die Datei nicht existiert
    shap_values_dtc= explainer_dtc(corpus_test)

    # Speichern Sie die SHAP-Werte in einer Datei
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values_dtc, f)
else:
    # Laden Sie die SHAP-Werte aus der Datei, wenn sie existiert
    with open(shap_values_file, "rb") as f:
        shap_values_dtc = pickle.load(f)

# 5.1 Investigate Biggest Misclassifications


In [ ]:
dtc_misclassifications = get_misclassifications(y_test, DTC_PREDICTION_FILE)


In [ ]:

print("total samples", len(y_test))
print("total misclassifications", len(dtc_misclassifications))

# count number of samples for label 1 and label 0
label_0 = y_test.count(0)
label_1 = y_test.count(1)



# print number of samples for label 1 and label 0

print("Number of samples for label 0: ", label_0)
print("Number of samples for label 1: ", label_1)

# count missclassifications for class 0 and for class 1

dtc_misclassifications_0 = dtc_misclassifications[(dtc_misclassifications['label'] == 0)]
dtc_misclassifications_1 = dtc_misclassifications[(dtc_misclassifications['label'] == 1)]

# print the number of missclassifications for class 0 and for class 1
print("Number of missclassifications for class 0: ", len(dtc_misclassifications_0))
print("Number of missclassifications for class 1: ", len(dtc_misclassifications_1))


# calculate the percentage of missclassifications for class 0 and for class 1
percentage_0 = len(dtc_misclassifications_0) / label_0
percentage_1 = len(dtc_misclassifications_1) / label_1

# print the percentage of missclassifications for class 0 and for class 1
print("Percentage of missclassifications for class 0: ", percentage_0)
print("Percentage of missclassifications for class 1: ", percentage_1)

In [ ]:
dtc_misclassifications[:10]

# 5.1.2 Most Wrong Positive classification

In [ ]:
most_wrong_positive_index = 1073
explanation = shap_values_dtc[most_wrong_positive_index, :, 1]
print(dtc_misclassifications.loc[most_wrong_positive_index])
print(corpus_test[most_wrong_positive_index])


In [ ]:
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

#### interpretation:

Why is this a claim?

# 5.1.2 Most Wrong Negative

In [ ]:
most_wrong_negative_index = dtc_misclassifications[dtc_misclassifications['label'] == 0].index[0]
most_wrong_negative_index


In [ ]:
explanation = shap_values_dtc[most_wrong_negative_index, :, 1]
print(dtc_misclassifications.loc[most_wrong_negative_index])
print(corpus_test[most_wrong_negative_index])

In [ ]:
shap.plots.text(explanation)
shap.plots.waterfall(explanation)
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

### interpretation
Thats not a claim.

but the model learned that the word "that" is introduction for a claim that is not the case.
in this case its only an introduction for a study...

# 5.2 Global Effects

In [ ]:
custom_global_explanation(shap_values_dtc[:,:,1])

In [ ]:
custom_global_shap_distribution(shap_values_dtc[:,:,1], threshold=0.02)

### interpretation

you can see shap values that contributing to claim are way more above the threshold than the shap values contributing against a claim.

What are the impacts of this?

-> the models classifies way more as claim than as no-claim because it has way more words that contribute to class 1.





# 6. SHAP for BERT

In [ ]:
""" 

In summary, the reason your code is not using the GPU is that the SHAP library does not support GPU acceleration for BERT models. Unfortunately, there is no direct solution to this issue. You can try looking for alternative libraries or methods that support GPU acceleration for BERT or similar text models.
 """

# Performance Optimazation
masker = shap.maskers.Text(tokenizer=r"\W+") # this will create a basic whitespace tokenizer
# explainer_logreg = shap.Explainer(model_logregression_new.predict_proba , masker)
explainer_bert = shap.Explainer(model_bert, masker)
import os.path
import pickle

shap_values_file = "shap_values_bert.pkl"

# Überprüfen, ob die Datei existiert
if not os.path.exists(shap_values_file):
    # Berechnen Sie die SHAP-Werte, wenn die Datei nicht existiert
    shap_values_bert= explainer_bert(corpus_test[:100])

    # Speichern Sie die SHAP-Werte in einer Datei
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values_bert, f)
else:
    # Laden Sie die SHAP-Werte aus der Datei, wenn sie existiert
    with open(shap_values_file, "rb") as f:
        shap_values_bert = pickle.load(f)

# 6.1 Investigate biggest Misclassifications

In [ ]:
dtc_misclassifications = get_misclassifications(y_test, BERT_PREDICTION_FILE)
dtc_misclassifications[:50]

## 6.1.2 Investigate biggest Misclassifications


In [ ]:
most_wrong_positive_index = 210

explanation = explainer_bert([corpus_test[most_wrong_positive_index]])
explanation = explanation[0, :, 1]

In [ ]:
print(dtc_misclassifications.loc[most_wrong_positive_index])
print(corpus_test[most_wrong_positive_index])
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=10)

### Interpretation

This is a claim but BERT fails to classify it as such:
claim: death warns against such misplaced values and condemns the practice of censorship as well as demonstrating there can be value in a show often dismissed as juvenile and immature
Prove: like south park or terrance and phillip ref

# 6.1.2 Biggest Negative misclassification

In [ ]:
most_wrong_negative_index = 1024

explanation = explainer_bert([corpus_test[most_wrong_negative_index]])
explanation = explanation[0, :, 1]

In [ ]:
print(dtc_misclassifications.loc[most_wrong_negative_index])
print(corpus_test[most_wrong_negative_index])
shap.plots.text(explanation)
shap.plots.waterfall(explanation)

In [ ]:
highlight_signal_words(explanation,round_shap_values=3,top_words=4)

### interpretation

The model learned that "study concluded" are words that lead to a claim.
I think its not labeled as a claim because its a neutral view on a study

# 6.2 Global Explanation



In [ ]:
custom_global_explanation(shap_values_bert[:,:,1])

In [ ]:
custom_global_shap_distribution(shap_values_bert[:,:,1], threshold=0.001)

# 7 Model Comparison

In [ ]:
from Sentiment.disagreements import get_disagreements

dtc_bert_sorted_by_diff,logreg_bert_sorted_by_diff,logreg_dtc_sorted_by_diff = get_disagreements(y_test,BERT_PREDICTION_FILE, LOGREG_PREDICTION_FILE, DTC_PREDICTION_FILE)

In [ ]:
dtc_bert_sorted_by_diff.head(10)

In [ ]:
logreg_bert_sorted_by_diff.head(10)

In [ ]:
# sample 1253 is intresting because there is a big differenc between the predictions of the three models

In [ ]:
bert_dtc_log_reg_biggest_difference_index = 1253
corpus_test[bert_dtc_log_reg_biggest_difference_index]

In [ ]:
dtc_explanation = explainer_dtc([corpus_test[bert_dtc_log_reg_biggest_difference_index]])
logreg_explanation = explainer_logreg([corpus_test[bert_dtc_log_reg_biggest_difference_index]])

In [ ]:
print('DTC Explanation')
highlight_signal_words(dtc_explanation[0,:,1],round_shap_values=3,top_words=6)
shap.plots.text(dtc_explanation[0,:,1])

In [ ]:
print('logreg Explanation')
highlight_signal_words(logreg_explanation[0,:,1],round_shap_values=3,top_words=6)
shap.plots.text(logreg_explanation[0,:,1])

In [ ]:
bert_explanation = explainer_bert([corpus_test[bert_dtc_log_reg_biggest_difference_index]])
bert_explanation = bert_explanation[0, :, 1]
print('BERT Explanation')
highlight_signal_words(bert_explanation,round_shap_values=3,top_words=6)
shap.plots.text(bert_explanation)
shap.plots.waterfall( bert_explanation)

# Interpretation

why is this a claim